- **u.data**: Chứa toàn bộ các đánh giá của 943 người dùng cho 1682 bộ phim. Mỗi người dùng đánh giá ít nhất 20 movie. Thông tin về thời điểm đánh giá cũng được cho nhưng chúng ta không sử dụng trong ví dụ này.
- **a.base**, **ua.test**, **ub.base**, **ub.test**: là hai cách chia toàn bộ dữ liệu ra thành hai tập con, một cho huấn luyện, một cho kiểm thử. Chúng ta sẽ thực hành trên ua.base và ua.test.
- **u.user**: Chứa thông tin về người dùng, bao gồm: id, tuổi, giới tính, nghề nghiệp, zipcode (vùng miền), vì những thông tin này cũng có thể ảnh hưởng tới sở thích của các người dùng. Tuy nhiên, trong ví dụ này, chúng ta sẽ không sử dụng các thông tin này, trừ thông tin về id để xác định các user khác nhau.
-  **u.genre**: Chứa tên của 19 thể loại phim. Các thể loại bao gồm: unknown, Action, Adventure, Animation, Children‘s, Comedy, Crime, Documentary, Drama, Fantasy, Film−Noir, Horror, Musical, Mystery, Romance, Sci−Fi, Thriller, War, Western,
- **u.item**: thông tin về mỗi bộ phim. Ví dụ:

  1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
  |0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0  
  2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
  |0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0  
  3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)  
  |0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0  
  4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)  
  |0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0

- Trong mỗi dòng, chúng ta sẽ thấy id của phim, tên phim, ngày phát hành, link trên imdb, và các số nhị phân 0, 1 phía cuối để chỉ ra bộ phim thuộc các thể loại nào trong 19 thể loại đã cho trong u.genre. Một bộ phim có thể thuộc nhiều thể loại khác nhau. Thông tin về thể loại này sẽ được dùng để xây dựng item profiles.

In [34]:
from __future__ import print_function
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

# Hai phương thức chính của class này là fit–tính ma trận similarity, và predict–dự đoán số
# sao mà một user sẽ đánh giá một item.
class uuCF(object):
  def __init__(self, Y_data, k, sim_func = cosine_similarity):
    self.Y_data = Y_data # a 2d array of shape (n_users, 3)
                         # each row of Y_data has form [user_id, item_id, rating]
    self.k = k # number of neighborhood
    self.sim_func = sim_func # similarity function, default: cosine_similarity
    self.Ybar = None # normalize data
    self.n_users = int(np.max(self.Y_data[:, 0])) + 1 # number of users
    self.n_items = int(np.max(self.Y_data[:, 1])) + 1 # number of items

  def fit(self):
    # normalized Y_data -> Ybar
    users = self.Y_data[:, 0] # all users - first column of Y_data
    self.Ybar = self.Y_data.copy()
    self.mu = np.zeros((self.n_users,)) # mảng 1 chiều với n_users số 0

    for n in range(self.n_users):
      # row indices of ratings made by user n
      ids = np.where(users == n)[0].astype(np.int32) # mảng

      # indices of all items rated by user n
      item_ids = self.Y_data[ids, 1]

      # ratings made by user n
      ratings = self.Y_data[ids, 2]

      # avoid zero division
      self.mu[n] = np.mean(ratings) if ids.size > 0 else 0
      self.Ybar[ids, 2] = ratings - self.mu[n] # normalized Y

    ## form the rating matrix as a sparse matrix.
    # see more: https://goo.gl/i2mmT2
    self.Ybar = sparse.coo_matrix((self.Ybar[:, 2],
                                  (self.Ybar[:, 1], self.Ybar[:, 0])),
                                  (self.n_items, self.n_users)).tocsr()
    self.S = self.sim_func(self.Ybar.T, self.Ybar.T)

  def pred(self, u, i):
    """predict the rating of user u for item i use RMSE - root mean square error"""
    # find item i
    ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)

    # all users who rated i
    users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)

    # similarity of u and users who rated i
    sim = self.S[u, users_rated_i]

    # most k similar users
    nns = np.argsort(sim)[-self.k:] # returns: k sorted index_array -> output: trả về k phần tử cuối cùng ( vì sort theo thứ tự tăng dần )
    nearest_s = sim[nns] # and the corresponding similarities

    # the corresponding ratings
    r = self.Ybar[i, users_rated_i[nns]]
    eps = 1e-8 # a small number to avoid zero division

    return (r*nearest_s).sum()/(np.abs(nearest_s).sum() + eps) + self.mu[u]


In [39]:
r_cols = ["user_id", "movie_id", "rating", "unix_timestamp"]
ratings_base = pd.read_csv("ml-100k/ua.base", sep="\t", names=r_cols)
ratings_test = pd.read_csv("ml-100k/ua.test", sep="\t", names=r_cols)

rate_train = ratings_base.values
rate_test = ratings_test.values

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

rs = uuCF(rate_train, k = 40)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
  pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
  SE += (pred - rate_test[n, 2])**2

RMSE = np.sqrt(SE/n_tests)
print("User-user CF, RMSE =", RMSE)

User-user CF, RMSE = 0.9766140289287265


In [42]:
# Tiếp theo, chúng ta áp dụng item-item CF vào tập cơ sở dữ liệu này. Để áp dụng item-item CF, chúng ta chỉ cần chuyển vị ma trận utility.
# Trong trường hợp này, vì ma trận utility được lưu dưới dạng [user_id, item_id, rating] nên ta chỉ cần đổi chỗ cột thứ nhất cho cột thứ hai của Y_data:
rate_train = rate_train[:, [1, 0, 2]]
rate_test = rate_test[:, [1, 0, 2]]

rs = uuCF(rate_train, k = 40)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
  pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
  SE += (pred - rate_test[n, 2])**2

RMSE = np.sqrt(SE/n_tests)
print("Item-item CF, RMSE =", RMSE)


Item-item CF, RMSE = 0.9688460838682366
